# Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import logit, probit, poisson, ols
import statsmodels.formula.api as smf
from pycaret.regression import *
from statsmodels.formula.api import ols
from pandas_profiling import ProfileReport
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.express as px
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML,Latex

# Helper Functions to create models

In [ ]:
def stat_model_eq(log_model,ex,target='Price_Rent_ratio_log'):
    p = [target]
    for j in ex:
        p.append(j)
    a = ''
    V = []
    cols = log_model.columns
    for i in cols:
        if i not in p:
            if i not in V:
                a += i + ' + '
    b = a[:-2]
    return b

def stat_model(log_model,ex=[],target='Price_Rent_ratio_log',normalize=True):
    b = stat_model_eq(log_model,ex,target)
    eq = target + ' ~ ' + b
    dvn = normalize_model(log_model,ex)
    if normalize:
        model = ols(eq,data=dvn)
        fitted_model = model.fit()
        #print(fitted_model.summary())
        
    else:
        model = ols(eq,data=log_model)
        fitted_model = model.fit()
        #print(fitted_model.summary())
        
    
    I = fitted_model.pvalues.index
    V = []
    for i in range(len(I)):
        v = fitted_model.pvalues[i]
        if v > .05:
            V.append(I[i])
            #print(I[i],v)
    
    return model
    
    
def normalize_model(df,ex):
    dfw = df.copy()
    p = ['obj_buildingType']
    for i in ex:
        p.append(i)
    var = [c for c in dfw.columns if c not in p]
    dv = dfw[var]
    dvn = (dv - dv.min())/dv.std()
    dvn['obj_buildingType'] = dfw['obj_buildingType']
    return dvn

def eq(di,ex=[],target='Price_Rent_ratio_log'):
    b = stat_model_eq(di,[],target)
    return target + ' ~ ' + b


def stat_model_eq_interaction_all(log_model,ex=[],target='Price_Rent_ratio_log'):
    p = [target]
    for j in ex:
        p.append(j)
    a = ''
    V = []
    cols = log_model.columns
    for i in cols:
        if i not in p:
            if i not in V:
                if i == 'obj_buildingType':
                    a += i + ' + '
                else:
                    a += 'C(obj_buildingType)*' + i + ' + '
    b = a[:-2]
    return b

def eq_all(di,ex=[],target='Price_Rent_ratio_log'):
    b = stat_model_eq_interaction_all(di)
    return target + ' ~ ' + b

# Importing the data

In [ ]:
dc = pd.read_csv('all_median_pr_merged_with_no_percent_for_elasticity.csv')
mc = dc.drop('zip',axis=1)

di = mc.copy()
di['public_transport'].replace(0,1,inplace=True)
log_col = [c for c in di.columns if c!= 'obj_buildingType']
log_model = pd.DataFrame()
for i in log_col:
    log_model[i+'_log'] = np.log(di[i])

log_model['obj_buildingType'] = di['obj_buildingType']
log_model.drop(['price_sup_log','rent_supply_log','North_of_city_log','East_of_city_log','sell_buy_ratio_log','population_18_30_log','population_30_50_log','bus_log','Wage_and_income_tax_log','Wage_and_income_taxpayers_log'],axis=1,inplace=True)

# Removing the category of houses that have less than 20 data points for reducing the overfitting

In [ ]:
count = log_model.groupby(['obj_buildingType']).count()[['Price_Rent_ratio_log']]
ag = log_model.groupby(['obj_buildingType']).mean()[['Price_Rent_ratio_log','rooms_log']]
ag['count'] = count
ag[ag['count'] > 20]
btype = ag[ag['count'] > 20].index
df = log_model[log_model['obj_buildingType'].isin(btype)]
df['area_per_room_log'] = df['livingSpace_log']-df['rooms_log']

df.drop('rooms_log',axis=1,inplace=True)
df.drop(['Lat_log','Lng_log'],axis=1,inplace=True)

dvn = df.copy()
equ = eq(df)

# Normalization is not required in log-log models

# Null Model

In [ ]:
model = ols('Price_Rent_ratio_log ~ 1',data=dvn)
null_model = model.fit()
null_model.summary()

# Helper functions to do statistical tests

In [ ]:
from statsmodels.stats.anova import anova_lm
def mcf(m1,null=null_model):
    a = (m1.llf - m1.df_model)/null.llf
    return 1 - a

def horowitz(m1,m2,null=null_model):
    ro1 = mcf(m1,null)
    ro2 = mcf(m2,null)
    
    print('mcf ro model 1: ' ,ro1)
    print('mcf ro model 2: ',ro2)
    
    
    
    if ro1 > ro2:
        rho = ro1
        rlo = ro2
        print('model 1 has higher mcf')
        deltadf = m1.df_model - m2.df_model
        
        v = -2*(rho-rlo)*null.llf + deltadf
        print(deltadf)
        return -np.sqrt(v)
    else:
        rho = ro2
        rlo = ro1
        print('model 2 has higher mcf')
        deltadf = m2.df_model - m1.df_model
        
        v = -2*(rho-rlo)*null.llf + deltadf
        return -np.sqrt(v)

def f_test(m1,m2):
    num = (m1.ssr - m2.ssr)/(m2.df_model - m1.df_model)
    den = m2.ssr/(m2.nobs - m2.df_model)
    d1 = (m2.df_model - m1.df_model)
    d2 = (m2.nobs - m2.df_model)
    return num/den,d1,d2

def stat_model_eq_interaction_all(log_model,ex=[],target='Price_Rent_ratio_log'):
    p = [target]
    for j in ex:
        p.append(j)
    a = ''
    V = []
    cols = log_model.columns
    for i in cols:
        if i not in p:
            if i not in V:
                if i == 'obj_buildingType':
                    a += i + ' + '
                else:
                    a += 'C(obj_buildingType)*' + i + ' + '
    b = a[:-2]
    return b

def eq_all(di,ex=[],target='Price_Rent_ratio_log'):
    b = stat_model_eq_interaction_all(di)
    return target + ' ~ ' + b

# Model with all variables

In [ ]:
model = ols(eq(df),data=dvn)
basic_model = model.fit()
model.fit().summary()

# all interaction

In [ ]:
model = ols(eq_all(df),data=dvn)
interaction_all_building = model.fit()
interaction_all_building.summary()

# Helper function to drop in-significant interactions using ANOVA

In [ ]:
def drop_interaction(df,interaction_all_building,target='Price_Rent_ratio',dvn=dvn):
    col_interact = [c for c in df.columns if c not in [target,'obj_buildingType']]
    rej = []
    for i in col_interact:
        e = ''
        for j in col_interact:
            if i==j:
                e += j + ' + '
            else:
                e += 'C(obj_buildingType)*' + j + ' + '
        a = e[:-2]
        b = target + ' ~ C(obj_buildingType) + ' + a
        model_i = ols(b,data=dvn)
        imodel = model_i.fit()
        p = anova_lm(imodel,interaction_all_building)['Pr(>F)'].iloc[1]
        if p > .05:
            print('reject the interaction with ',i)
            rej.append(i)
    
    rej.append('obj_buildingType')
    ne = ''
    for i in col_interact:
        if i not in rej:
            ne += 'C(obj_buildingType)*' + i + ' + '
        else:
            ne += i + ' + '
    m = target + ' ~ C(obj_buildingType) + ' + ne[:-2]
    return m,rej

# Removing insignificant interactions using ANOVA

In [ ]:
m,rej = drop_interaction(df,interaction_all_building,target='Price_Rent_ratio_log')

# Model

In [ ]:
model = ols(m,data=dvn)
interaction_selected = model.fit()
interaction_selected.summary()

In [ ]:
I = interaction_selected.pvalues.index
V = []
for i in range(len(I)):
    v = interaction_selected.pvalues[i]
    if v > .1:
        V.append(I[i])
        print(I[i],v)

# Significant Final Model 

In [ ]:
model = ols('Price_Rent_ratio_log ~ C(obj_buildingType) + C(obj_buildingType)*Deutsche_log  + C(obj_buildingType)*GDP_Per_Capita_state_log + Gross_domestic_product_per_person_in_employment_log + Population_State_log + Total_amount_of_income_log + cafe_log + doctors_log + fast_food_log + gross_domestic_product_log + restaurant_log + zip_population_log + C(obj_buildingType)*age_log + C(obj_buildingType)*livingSpace_log + avg_space_per_room_log + population_density_state_log + dist_aprox_log ',data=dvn)
interaction_selected_final = model.fit()
interaction_selected_final.summary()

# Results interpretation

In [ ]:
interaction_selected_final.summary()

# Data Frame of coefficients

In [ ]:
import plotly.graph_objects as go

dp = pd.DataFrame()
Param = []
Coef = []
P_V = []

for i in range(len(interaction_selected.params.values)):
    param = interaction_selected.params.index[i]
    coef = interaction_selected.params.values[i]
    p_v = interaction_selected.pvalues[i]
    
    if p_v < .1:
        Param.append(param)
        Coef.append(coef)
        P_V.append(p_v)
dp['Parameters'] = Param
dp['Coef'] = Coef
dp['p-values'] = P_V
dp['p-values'] = dp['p-values'].round(3)
dp.sort_values('p-values')

In [ ]:
de = dp.copy()
de['st'] = de['Parameters'].apply(lambda x: 1 if 'C(obj_buildingType)' in x else 0)
de = de[de['st'] == 1]

de['nst'] = de['Parameters'].apply(lambda x: 1 if 'livingSpace' not in x else 0)
de = de[de['nst'] == 1]
de.drop('nst',axis=1,inplace=True)

de['nst'] = de['Parameters'].apply(lambda x: 1 if 'Lng' not in x else 0)
de = de[de['nst'] == 1]
de.drop('nst',axis=1,inplace=True)

de['nst'] = de['Parameters'].apply(lambda x: 1 if 'age' not in x else 0)
de = de[de['nst'] == 1]
de.drop('nst',axis=1,inplace=True)

de['nst'] = de['Parameters'].apply(lambda x: 1 if 'rooms' not in x else 0)
de = de[de['nst'] == 1]
de.drop('nst',axis=1,inplace=True)

de['nst'] = de['Parameters'].apply(lambda x: 1 if 'GDP' not in x else 0)
de = de[de['nst'] == 1]
de.drop('nst',axis=1,inplace=True)

In [ ]:
report = pd.DataFrame()
models_report = [null_model,basic_model,interaction_all_building,interaction_selected_final]
R2 = []
AR2 = []
AIC = []
DF = []

for i in models_report:
    R2.append(i.rsquared)
    AR2.append(i.rsquared_adj)
    AIC.append(i.aic)
    DF.append(i.df_model)

Models = ['Null Model - log','Basic Model - log','All Interactions with building - Log','Interaction selected - log']
report['Models'] = Models
report['R2'] = R2
report['Adj R2'] = AR2
report['AIC'] = AIC
report['DF'] = DF

report.to_csv('log_model_report.csv')

# Helper Functions to visualize the coefficients with p-values

In [ ]:
def plot_coef_cat(dp,cat):
    dr = dp.copy()
    
    if cat == '':
        cat = 'other features'
        dr['status'] = dr['Parameters'].apply(lambda x: 1 if 'C(obj_buildingType)' not in x else 0)
        dr = dr[dr['status'] == 1]
    
    else:
        dr['status'] = dr['Parameters'].apply(lambda x: 0 if cat not in x else 1)
        dr = dr[dr['status'] == 1]
    

    if cat != '':
        k = list(dr['Parameters'])
        if cat not in k:
            lc = 0
        if cat in k:
            lc = dr[dr['Parameters'] == cat]['Coef'].iloc[0]
        dr['new_Coef'] = dr['Coef'].apply(lambda x: x+lc if x!=lc else x)
    
    else:
        dr['new_Coef'] = dr['Coef']
    
    dr['categ'] = dr['Parameters'].apply(lambda x:  x.split('[T.')[1].split(']:')[0] if '[T.' in x else cat) 
    
    
    fig = px.bar(dr,x=dr['new_Coef'], y=dr['Parameters'],text=dr['new_Coef'].round(2),
            color='p-values',
            orientation='h'
             )
    
    fig.update_layout(
    title='Significant Impact on Price to Rent Ratio due to ' + cat,
    xaxis=dict(
        title='Coeficient in log model - Elasticity',
        titlefont_size=16,
        tickfont_size=14,
    )
    )
    
    fig.show()
    
    for j in list(dr['categ']):
        if j!= cat:
            scatter_cat(df,j,cat)
    

def scatter_cat(dr,cat,par):
    df = dr.copy()
    df = df[df['obj_buildingType'] == cat][['obj_buildingType',par,'Price_Rent_ratio_log']]
    
    fig = px.scatter(df, x=par, y='Price_Rent_ratio_log', trendline="ols")
    fig.update_layout(
    title='Scatter Plot - Price to Rent Ratio  vs ' + cat,
    
    )
    fig.show()
    

# Impact on type of houses

In [ ]:
fig = go.Figure(data=[go.Bar(
            x=de['Coef'], y=de['Parameters'],
            text=de['Coef'].round(2),
            textposition='outside',orientation='h',marker_color=de['Coef']
        )])

fig.update_layout(
    title='Significant Impact on Price to Rent Ratio due to Type of House',
    xaxis=dict(
        title='Log Model Coefficient',
        titlefont_size=16,
        tickfont_size=14,
    )
    )
        
fig.show()



# Visualization of all the results - Elasticity

In [ ]:
plot_coef_cat(dp,'')
plot_coef_cat(dp,'Deutsche_log')
plot_coef_cat(dp,'GDP_Per_Capita_state_log')
plot_coef_cat(dp,'livingSpace_log')
plot_coef_cat(dp,'age_log')

# Stargazer to compare model coefficients

In [ ]:
stargazer = Stargazer([null_model,basic_model,interaction_selected,interaction_selected_final])
HTML(stargazer.render_html())